## 问题定义

对待求解问题的数学描述如下：
![Equation](./resource/equation.png)

其中，$u$为待求解物理量，$x$为空间坐标，$t$为时间，$c=1$为波速

并且对于上述求解问题，解析解为：$u=\sin(x)(\sin(t) + \cos(t))$

### 求解目标

给定坐标$(x,t)$以及部分观测数据，要求求解波以及未知波速$c$


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
# 必要的符号运算
from sympy import Symbol, Eq, Abs, Function, Number, sin
import numpy as np

import modulus.sym

# 超参数
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.hydra.utils import compose

# 求解器
from modulus.sym.solver import Solver

# domain
from modulus.sym.domain import Domain

# 几何物体
from modulus.sym.geometry.primitives_1d import Line1D

# 约束
from modulus.sym.domain.constraint import (
    PointwiseConstraint
)

# validator
from modulus.sym.domain.monitor import PointwiseMonitor
from modulus.sym.domain.validator import PointwiseValidator

# inferencer
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key

# Equation
# 导入抽象基类
from modulus.sym.eq.pde import PDE

# post process
from modulus.sym.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)
import matplotlib.pyplot as plt

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 100000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: ${training.rec_results_freq}
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: fa

### 定义必要组件

#### PDE

本案例特殊之处在于需要自行定义PDE和边界条件

自定义PDE通过继承抽象基类(modulus.sym.eq.pde.PDE)实现

In [4]:
class WaveEquation1D(PDE):
    """
    Wave equation 1D
    The equation is given as an example for implementing
    your own PDE. A more universal implementation of the
    wave equation can be found by
    `from modulus.sym.eq.pdes.wave_equation import WaveEquation`.

    Parameters
    ==========
    c : float, string
        Wave speed coefficient. If a string then the
        wave speed is input into the equation.
    """

    name = "WaveEquation1D"

    def __init__(self, c=1.0):
        # 空间坐标
        x = Symbol("x")

        # 时间坐标
        t = Symbol("t")

        # 输入变量
        input_variables = {"x": x, "t": t}

        # 输出变量
        u = Function("u")(*input_variables)

        # 若传入的c为字符串，则表明c是随输入变量变化的量（inverse）
        # 否则定义为Number
        if type(c) is str:
            c = Function(c)(*input_variables)
        elif type(c) in [float, int]:
            c = Number(c)

        # 构建方程
        self.equations = {}
        self.equations["wave_equation"] = u.diff(t, 2) - (c**2 * u.diff(x)).diff(x)

we = WaveEquation1D(c='c')  # 定义c为未知参数，可优化

#### Model

In [5]:
# 输入为空间坐标和时间坐标
# 输出为u
wave_net = instantiate_arch(
        input_keys=[Key("x"), Key("t")],
        output_keys=[Key("u")],
        cfg=cfg.arch.fully_connected,
    )
print(wave_net)

# 输入为空间坐标和时间坐标
# 输出为c
invert_net = instantiate_arch(
    input_keys=[Key("x"), Key("t")],
    output_keys=[Key("c")],
    cfg=cfg.arch.fully_connected,
)

# 注意这里创建了detach
# 在Modulus中，detach的逻辑是：对于该模型，使用detach的输入，从而避免通过这些被detach的模型backward
# 更直观的例子可以参考06
nodes = (
    we.make_nodes(detach_names=["u__x", "u__x__x", "u__t__t"])
    + [wave_net.make_node(name="wave_network")]
    + [invert_net.make_node(name="invert_network")]
)

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)


#### Geo

在本问题中可以不定义Geo，因为Inverse会引入真实的观测数据，对这些观测坐标同时进行PDE监督和Data监督即可

#### Data

In [6]:
# prepare input data
L = float(np.pi)
deltaT = 0.01
deltaX = 0.01
x = np.arange(0, L, deltaX)
t = np.arange(0, 2 * L, deltaT)
X, T = np.meshgrid(x, t)
X = np.expand_dims(X.flatten(), axis=-1)
T = np.expand_dims(T.flatten(), axis=-1)
u = np.sin(X) * (np.cos(T) + np.sin(T))
invar_numpy = {"x": X, "t": T}
outvar_numpy = {"u": u}
outvar_numpy["wave_equation"] = np.zeros_like(outvar_numpy["u"])

#### Domain

在Domain中定义约束以及训练所需的各种组件

In [7]:
# make domain
domain = Domain()

#### 边界条件

本例可以不使用边界条件

#### 初始条件

本例可以不使用初始条件

#### PDE约束

所有的观测数据点同样满足PDE约束

In [8]:
# data and pde loss
data = PointwiseConstraint.from_numpy(
    nodes=nodes,
    invar=invar_numpy,
    outvar=outvar_numpy,
    batch_size=cfg.batch_size.data,
)
domain.add_constraint(data, "interior_data")

验证器以及其他必要组件

In [9]:
# 针对c的monitors
monitor = PointwiseMonitor(
    invar_numpy,
    output_names=["c"],
    metrics={"mean_c": lambda var: torch.mean(var["c"])},
    nodes=nodes,
)
domain.add_monitor(monitor)

In [10]:
# 创建Validator
validator = PointwiseValidator(
    nodes=nodes, invar={"x": X, "t": T}, true_outvar={"u": u}, batch_size=128, plotter=ValidatorPlotter()
)
domain.add_validator(validator)

### 求解器以及求解

In [11]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [12]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [13]:
slv.solve()

2024-02-27 07:39:39,712 - modulus.sym.trainer - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-02-27 07:39:39,722 - modulus.sym.trainer - INFO - attempting to restore from: /workspace/07_1D_Wave_Inverse/outputs
2024-02-27 07:39:39,723 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-02-27 07:39:39,724 - modulus.sym.trainer - WARNING - model invert_network.0.pth not found
2024-02-27 07:39:39,724 - modulus.sym.trainer - WARNING - model wave_network.0.pth not found
2024-02-27 07:39:41,021 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  1.057e-01s
2024-02-27 07:39:51,934 - modulus.sym.trainer - INFO - [step:          0] record validators time:  1.091e+01s
2024-02-27 07:39:51,987 - modulus.sym.trainer - INFO - [step:          0] record monitor time:  1.998e-02s
2024-02-27 07:39:52,048 - modulus.sym.trainer - INFO - [step:          0] saved

2024-02-27 07:43:11,911 - modulus.sym.trainer - INFO - [step:       4300] loss:  6.709e+00, time/iteration:  3.399e+01 ms
2024-02-27 07:43:15,347 - modulus.sym.trainer - INFO - [step:       4400] loss:  7.450e+00, time/iteration:  3.435e+01 ms
2024-02-27 07:43:18,735 - modulus.sym.trainer - INFO - [step:       4500] loss:  7.254e+00, time/iteration:  3.387e+01 ms
2024-02-27 07:43:22,097 - modulus.sym.trainer - INFO - [step:       4600] loss:  3.654e+00, time/iteration:  3.360e+01 ms
2024-02-27 07:43:25,476 - modulus.sym.trainer - INFO - [step:       4700] loss:  1.191e+01, time/iteration:  3.376e+01 ms
2024-02-27 07:43:28,847 - modulus.sym.trainer - INFO - [step:       4800] loss:  5.057e+00, time/iteration:  3.369e+01 ms
2024-02-27 07:43:32,226 - modulus.sym.trainer - INFO - [step:       4900] loss:  8.551e+00, time/iteration:  3.378e+01 ms
2024-02-27 07:43:35,666 - modulus.sym.trainer - INFO - [step:       5000] record constraint batch time:  9.337e-02s
2024-02-27 07:43:45,759 - modu

2024-02-27 07:46:50,972 - modulus.sym.trainer - INFO - [step:       9200] loss:  4.783e+00, time/iteration:  3.217e+01 ms
2024-02-27 07:46:54,324 - modulus.sym.trainer - INFO - [step:       9300] loss:  1.251e+01, time/iteration:  3.351e+01 ms
2024-02-27 07:46:57,693 - modulus.sym.trainer - INFO - [step:       9400] loss:  1.479e+01, time/iteration:  3.367e+01 ms
2024-02-27 07:47:01,059 - modulus.sym.trainer - INFO - [step:       9500] loss:  7.269e+00, time/iteration:  3.364e+01 ms
2024-02-27 07:47:04,416 - modulus.sym.trainer - INFO - [step:       9600] loss:  4.773e+01, time/iteration:  3.356e+01 ms
2024-02-27 07:47:07,765 - modulus.sym.trainer - INFO - [step:       9700] loss:  2.888e+00, time/iteration:  3.348e+01 ms
2024-02-27 07:47:11,119 - modulus.sym.trainer - INFO - [step:       9800] loss:  6.019e+00, time/iteration:  3.352e+01 ms
2024-02-27 07:47:14,473 - modulus.sym.trainer - INFO - [step:       9900] loss:  1.673e+01, time/iteration:  3.352e+01 ms
2024-02-27 07:47:17,933 

2024-02-27 07:50:25,200 - modulus.sym.trainer - INFO - [step:      14100] loss:  3.288e+01, time/iteration:  3.246e+01 ms
2024-02-27 07:50:28,407 - modulus.sym.trainer - INFO - [step:      14200] loss:  2.770e+01, time/iteration:  3.206e+01 ms
2024-02-27 07:50:31,744 - modulus.sym.trainer - INFO - [step:      14300] loss:  2.742e+01, time/iteration:  3.336e+01 ms
2024-02-27 07:50:35,132 - modulus.sym.trainer - INFO - [step:      14400] loss:  2.061e+01, time/iteration:  3.386e+01 ms
2024-02-27 07:50:38,654 - modulus.sym.trainer - INFO - [step:      14500] loss:  1.963e+01, time/iteration:  3.521e+01 ms
2024-02-27 07:50:42,003 - modulus.sym.trainer - INFO - [step:      14600] loss:  1.779e+01, time/iteration:  3.347e+01 ms
2024-02-27 07:50:45,396 - modulus.sym.trainer - INFO - [step:      14700] loss:  1.656e+01, time/iteration:  3.392e+01 ms
2024-02-27 07:50:48,924 - modulus.sym.trainer - INFO - [step:      14800] loss:  1.553e+01, time/iteration:  3.526e+01 ms
2024-02-27 07:50:52,295 

2024-02-27 07:54:01,827 - modulus.sym.trainer - INFO - [step:      19000] loss:  1.483e+00, time/iteration:  1.356e+02 ms
2024-02-27 07:54:05,297 - modulus.sym.trainer - INFO - [step:      19100] loss:  2.884e+00, time/iteration:  3.469e+01 ms
2024-02-27 07:54:08,648 - modulus.sym.trainer - INFO - [step:      19200] loss:  1.948e+00, time/iteration:  3.349e+01 ms
2024-02-27 07:54:11,999 - modulus.sym.trainer - INFO - [step:      19300] loss:  7.056e+00, time/iteration:  3.350e+01 ms
2024-02-27 07:54:15,219 - modulus.sym.trainer - INFO - [step:      19400] loss:  3.460e+00, time/iteration:  3.218e+01 ms
2024-02-27 07:54:18,579 - modulus.sym.trainer - INFO - [step:      19500] loss:  1.951e+00, time/iteration:  3.359e+01 ms
2024-02-27 07:54:22,063 - modulus.sym.trainer - INFO - [step:      19600] loss:  4.107e+00, time/iteration:  3.481e+01 ms
2024-02-27 07:54:25,590 - modulus.sym.trainer - INFO - [step:      19700] loss:  1.518e+00, time/iteration:  3.526e+01 ms
2024-02-27 07:54:28,986 

...

2024-02-27 08:37:06,275 - modulus.sym.trainer - INFO - [step:      78100] loss:  3.132e-02, time/iteration:  3.488e+01 ms
2024-02-27 08:37:09,870 - modulus.sym.trainer - INFO - [step:      78200] loss:  5.566e-02, time/iteration:  3.594e+01 ms
2024-02-27 08:37:13,439 - modulus.sym.trainer - INFO - [step:      78300] loss:  4.288e-02, time/iteration:  3.568e+01 ms
2024-02-27 08:37:16,822 - modulus.sym.trainer - INFO - [step:      78400] loss:  2.238e-02, time/iteration:  3.382e+01 ms
2024-02-27 08:37:20,149 - modulus.sym.trainer - INFO - [step:      78500] loss:  2.472e-02, time/iteration:  3.325e+01 ms
2024-02-27 08:37:23,350 - modulus.sym.trainer - INFO - [step:      78600] loss:  2.592e-02, time/iteration:  3.200e+01 ms
2024-02-27 08:37:26,567 - modulus.sym.trainer - INFO - [step:      78700] loss:  2.681e-02, time/iteration:  3.216e+01 ms
2024-02-27 08:37:29,876 - modulus.sym.trainer - INFO - [step:      78800] loss:  5.680e-02, time/iteration:  3.306e+01 ms
2024-02-27 08:37:33,223 

2024-02-27 08:40:57,677 - modulus.sym.trainer - INFO - [step:      83000] loss:  2.279e-02, time/iteration:  1.362e+02 ms
2024-02-27 08:41:01,083 - modulus.sym.trainer - INFO - [step:      83100] loss:  3.344e-02, time/iteration:  3.405e+01 ms
2024-02-27 08:41:04,451 - modulus.sym.trainer - INFO - [step:      83200] loss:  4.095e-02, time/iteration:  3.364e+01 ms
2024-02-27 08:41:07,814 - modulus.sym.trainer - INFO - [step:      83300] loss:  2.504e-02, time/iteration:  3.361e+01 ms
2024-02-27 08:41:11,363 - modulus.sym.trainer - INFO - [step:      83400] loss:  2.840e-02, time/iteration:  3.548e+01 ms
2024-02-27 08:41:14,746 - modulus.sym.trainer - INFO - [step:      83500] loss:  1.976e-02, time/iteration:  3.381e+01 ms
2024-02-27 08:41:18,148 - modulus.sym.trainer - INFO - [step:      83600] loss:  2.317e-02, time/iteration:  3.400e+01 ms
2024-02-27 08:41:21,573 - modulus.sym.trainer - INFO - [step:      83700] loss:  2.242e-02, time/iteration:  3.423e+01 ms
2024-02-27 08:41:25,967 

2024-02-27 08:45:08,897 - modulus.sym.trainer - INFO - [step:      88000] saved checkpoint to /workspace/07_1D_Wave_Inverse/outputs
2024-02-27 08:45:08,899 - modulus.sym.trainer - INFO - [step:      88000] loss:  2.751e-02, time/iteration:  1.454e+02 ms
2024-02-27 08:45:13,329 - modulus.sym.trainer - INFO - [step:      88100] loss:  3.634e-02, time/iteration:  4.430e+01 ms
2024-02-27 08:45:17,566 - modulus.sym.trainer - INFO - [step:      88200] loss:  1.941e-02, time/iteration:  4.235e+01 ms
2024-02-27 08:45:21,997 - modulus.sym.trainer - INFO - [step:      88300] loss:  3.471e-02, time/iteration:  4.429e+01 ms
2024-02-27 08:45:26,384 - modulus.sym.trainer - INFO - [step:      88400] loss:  2.549e-02, time/iteration:  4.384e+01 ms
2024-02-27 08:45:30,720 - modulus.sym.trainer - INFO - [step:      88500] loss:  2.176e-02, time/iteration:  4.334e+01 ms
2024-02-27 08:45:35,180 - modulus.sym.trainer - INFO - [step:      88600] loss:  2.827e-02, time/iteration:  4.458e+01 ms
2024-02-27 08:

2024-02-27 08:49:22,075 - modulus.sym.trainer - INFO - [step:      93000] record validators time:  1.036e+01s
2024-02-27 08:49:22,097 - modulus.sym.trainer - INFO - [step:      93000] record monitor time:  1.709e-02s
2024-02-27 08:49:22,160 - modulus.sym.trainer - INFO - [step:      93000] saved checkpoint to /workspace/07_1D_Wave_Inverse/outputs
2024-02-27 08:49:22,161 - modulus.sym.trainer - INFO - [step:      93000] loss:  1.808e-02, time/iteration:  1.508e+02 ms
2024-02-27 08:49:26,543 - modulus.sym.trainer - INFO - [step:      93100] loss:  2.687e-02, time/iteration:  4.381e+01 ms
2024-02-27 08:49:30,207 - modulus.sym.trainer - INFO - [step:      93200] loss:  2.215e-02, time/iteration:  3.662e+01 ms
2024-02-27 08:49:33,737 - modulus.sym.trainer - INFO - [step:      93300] loss:  1.578e-02, time/iteration:  3.527e+01 ms
2024-02-27 08:49:37,222 - modulus.sym.trainer - INFO - [step:      93400] loss:  2.233e-02, time/iteration:  3.482e+01 ms
2024-02-27 08:49:40,597 - modulus.sym.tra

2024-02-27 08:53:13,163 - modulus.sym.trainer - INFO - [step:      98000] record constraint batch time:  9.921e-02s
2024-02-27 08:53:22,819 - modulus.sym.trainer - INFO - [step:      98000] record validators time:  9.654e+00s
2024-02-27 08:53:22,838 - modulus.sym.trainer - INFO - [step:      98000] record monitor time:  1.632e-02s
2024-02-27 08:53:22,899 - modulus.sym.trainer - INFO - [step:      98000] saved checkpoint to /workspace/07_1D_Wave_Inverse/outputs
2024-02-27 08:53:22,901 - modulus.sym.trainer - INFO - [step:      98000] loss:  1.929e-02, time/iteration:  1.313e+02 ms
2024-02-27 08:53:26,244 - modulus.sym.trainer - INFO - [step:      98100] loss:  1.901e-02, time/iteration:  3.342e+01 ms
2024-02-27 08:53:29,629 - modulus.sym.trainer - INFO - [step:      98200] loss:  2.026e-02, time/iteration:  3.383e+01 ms
2024-02-27 08:53:33,345 - modulus.sym.trainer - INFO - [step:      98300] loss:  2.375e-02, time/iteration:  3.713e+01 ms
2024-02-27 08:53:36,903 - modulus.sym.trainer -

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

如果使用了PointwiseValidator则可以直接查看验证的结果：

![u](./outputs/validators/validator_u.png)

In [14]:
import pandas as pd

# raed mean_c
mean_c = pd.read_csv('./outputs/monitors/mean_c.csv')
mean_c

,step,mean_c
0,0,-4.306354
1,1000,0.957068
2,2000,0.977765
3,3000,0.984083
4,4000,0.990973
...,...,...
96,96000,0.996833
97,97000,0.996408
98,98000,0.996908
99,99000,0.996659
